In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [12]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle


%config Completer.use_jedi = False 




#--- Notebook run settings
run = 'command line' # 'manual' 'command line'
#run = 'manual'


# Settings for the multiple linear regression model (MLR)
#--- Arguments
if run == 'command line':
    # Read command line arguments
    arguments = os.environ['NB_ARGS']
    alpha, date_exp_str = arguments.split(',')
    
    alpha              = float(alpha)   
    
elif run == 'manual':
    # Enter arguments manually 
    alpha              = 0.01
    date_exp_str       = '230417_TempSS_FMort'
    remove_data        = True

print(date_exp_str, alpha)    

230417_TempSS_FMort 0.01


In [13]:
import glob

file_list = glob.glob('site_reg_results/'+date_exp_str+'_*'+str(alpha)+'*_SITE_best_Models_dAIC.pkl')
file_list = [f for f in file_list if 'False' not in f]

if ('TempSS_FMort' in date_exp_str):
    file_list = [f for f in file_list if 'TempSS_FMort' in f]
    file_list = [f for f in file_list if 'No' in f]
else:
    file_list = [f for f in file_list if 'TempSS_FMort' not in f]
    
file_list

['site_reg_results/230417_TempSS_FMort_NoInsectsFire_0.01_True_SITE_best_Models_dAIC.pkl',
 'site_reg_results/230417_TempSS_FMort_NoFire_0.01_True_SITE_best_Models_dAIC.pkl']

In [14]:
#---------------#
#  Import data  #
#---------------#
mtf_type = ['count','cmass']
mortality_groups = ['M_All','M_Fire','M_NoFire','M_Other']


# Initialise storage
res_tables  = np.empty((len(mortality_groups),len(mtf_type)),dtype='object') # table; res_objects
df_res_mort = pd.DataFrame()
for f,file in enumerate(file_list):
    
    with open(file,'rb') as l:
        res_temp = pickle.load(l)
        
        for t in range(2):
            res_tables[f,t] = res_temp[t]

            
        res_tables_formatted = res_tables.copy()
        df_mort = pd.DataFrame()
        for t,tpe in enumerate(mtf_type):
            res_tables_formatted[f,t] = res_tables[f,t].set_index(['model','climate','Mortality','N_obs'])
            res_tables_formatted[f,t] = res_tables_formatted[f,t][['r2','AIC','d_AIC','RMSE','ID']].copy()
            res_tables_formatted[f,t].columns = pd.MultiIndex.from_tuples([(i,tpe) for i in 
                                                                           res_tables_formatted[f,t].columns])
        
        df_mort = pd.concat(res_tables_formatted[f], axis=1)
        df_mort = df_mort.reset_index(drop=False)
        df_mort = df_mort[['Mortality','model','r2','AIC','d_AIC','RMSE','ID','N_obs','climate']].copy()
        df_melt = df_mort.melt(ignore_index=False)
        idx_r2  = df_melt[df_melt.variable_0 == 'r2'].sort_values('value',ascending=False).index.unique()
        df_mort = df_mort.reindex(idx_r2).reset_index(drop=True).copy()
        df_mort.set_index(['Mortality','model'],inplace=True)
        df_res_mort = pd.concat([df_res_mort,df_mort],axis=0)

# Sort by mortality group
df_res_mort = df_res_mort.reset_index(drop=False)
df_res_mort.Mortality = df_res_mort.Mortality.astype("category")
df_res_mort.Mortality = df_res_mort.Mortality.cat.set_categories(mortality_groups)
df_res_mort['Level'] = 'Site' # Add analysis level
       
df_res_mort = df_res_mort.sort_values(["Mortality"]).set_index(['Level','Mortality','model'])
display(df_res_mort)
with open('site_reg_results/'+date_exp_str+'_'+str(alpha)+'_SITE_best_models_dAIC.pkl','wb') as f:
    pickle.dump(df_res_mort,f)

r2         AIC       d_AIC        RMSE        \
                              count cmass count cmass count cmass count cmass   
Level Mortality model                                                           
Site  M_NoFire  MTF ~ MATsoil  0.24  0.29  91.1  93.5     0     0  25.6  50.4   
                MTF ~ MAT      0.21  0.25  92.6  95.9   1.5   2.4  25.1  49.4   
      M_Other   MTF ~ MATsoil  0.22  0.28  67.1  69.6     0     0  18.7  37.6   
                MTF ~ MAT       NaN  0.23   NaN  74.7   NaN   5.1   NaN  55.2   

                                 ID       N_obs climate  
                              count cmass                
Level Mortality model                                    
Site  M_NoFire  MTF ~ MATsoil     0     1  38.0     NaN  
                MTF ~ MAT         5     3  38.0     NaN  
      M_Other   MTF ~ MATsoil     2     3  29.0     NaN  
                MTF ~ MAT       NaN     7  28.0     NaN